In [1]:
import numpy as np
import os
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
import pickle 

file_name = 'binary_search_val_msgs.pkl'
with open(file_name, 'rb') as f:
    batched_msgs = pickle.load(f)

In [3]:
batched_msgs.shape

(31464, 320)

In [13]:
std_devs = np.array([np.std(batched_msgs[:,i]) for i in range(100)])
print(std_devs)
sort_idx = np.argsort(-std_devs)
best_10_idx = sort_idx[:10]
best_idx = np.argmax(std_devs)
print("Best:", best_idx, ", std_dev:", std_devs[best_idx])
print("Best 10:", best_10_idx, ", std_devs:", std_devs[best_10_idx])

[0.01252689 0.00762447 0.00774388 0.00922599 0.00726951 0.00537647
 0.00499492 0.0099197  0.02389987 0.023113   0.00855329 0.01252523
 0.01380229 0.00437027 0.00469598 0.01421649 0.01667366 0.012155
 0.01566231 0.00479985 0.01285292 0.00879293 0.00541652 0.01384215
 0.00416283 0.00412787 0.00931411 0.00486765 0.01204427 0.02159762
 0.00875297 0.01016873 0.01413494 0.01031967 0.00998424 0.01553409
 0.0117391  0.0108287  0.00954389 0.00682799 0.00657129 0.02217012
 0.01966925 0.00273711 0.00825443 0.01134973 0.02308673 0.00820303
 0.00660286 0.0067474  0.01689328 0.01634214 0.00350134 0.0069412
 0.00877685 0.00802486 0.0121758  0.01061562 0.00656631 0.0102165
 0.02361109 0.00786607 0.00651675 0.0101824  0.20594607 0.35757607
 0.271627   0.363441   0.18785411 0.6017315  0.13898751 0.24175665
 0.26299733 0.09713102 0.4098434  0.15418614 0.14572759 0.5805387
 0.2919327  0.15503152 0.63866925 0.40523705 0.11090343 0.8240292
 0.22983459 0.39106402 0.61454076 0.26210502 0.22109796 0.47565815
 

In [8]:
best_message = np.argmax([np.std(batched_msgs[:,i]) for i in range(100)])

In [9]:
best_message

83

In [14]:
import numpy as np
import pickle as pkl
from pysr import PySRRegressor
import sympy

In [16]:
batched_msgs

array([[ 0.00033134, -0.00036411,  0.00042683, ...,  0.10840471,
         0.22307397, -0.02588965],
       [ 0.00033134, -0.00036411,  0.00042683, ...,  0.10840471,
         0.22307397, -0.02588965],
       [ 0.00033134, -0.00036411,  0.00042683, ...,  0.10840471,
         0.22307397, -0.02588965],
       ...,
       [ 0.00033134, -0.00036411,  0.00042683, ...,  0.0638169 ,
         0.14041297, -0.01902727],
       [ 0.00033134, -0.00036411,  0.00042683, ...,  0.0638169 ,
         0.14041297, -0.01902727],
       [-0.00183076, -0.0200084 ,  0.02849627, ...,  0.0638169 ,
         0.14041297, -0.01902727]], dtype=float32)

In [17]:
X = batched_msgs[:,best_10_idx[1:]]
y = batched_msgs[:,best_10_idx[0]]

print(X.shape, y.shape)

(31464, 9) (31464,)


In [18]:
model = PySRRegressor(
    model_selection="best",  # Result is mix of simplicity+accuracy
    niterations=100,
    binary_operators=[
        "plus", "sub", "mult", "div", "pow",
        "min", "max",
    ],
    unary_operators=[
        "exp", "log", "abs",
        "relu(x) = (x >= 0.0f3) ? x : 0.0f3",
        "heaviside(x) = x > 0.0f3 ? 1.0f3 : x == 0.0f3 ? 0.5f3 : 0.0f3",
    ],
    extra_sympy_mappings={
        "min": lambda x, y: sympy.Piecewise((x, x < y), (y, True)),
        "max": lambda x, y: sympy.Piecewise((y, x < y), (x, True)),
        "abs": lambda x: sympy.Piecewise((x, x >= 0.0), (-x, True)),
        "relu": lambda x: sympy.Piecewise((x, x >= 0.0), (0.0, True)),
        "heaviside": sympy.Heaviside,
    },
    complexity_of_operators={"pow":3, "exp":3, "log":3},
    # loss="loss(x, y) = (x - y)^2",
    loss="L1DistLoss()",
    maxsize=10,
    maxdepth=5
)

In [19]:
model.fit(X[:100], y[:100])

/home/kausta/miniconda3/envs/pysr/lib/python3.8/site-packages/pysr/sr.py:1257: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
